# Introduction to Faiss

## Imports

In [1]:
import numpy as np
import polars as pl

from sentence_transformers import SentenceTransformer
import faiss


## Introduction to Faiss

### Data Loading

In [ ]:
# data = [
#     "../data/2012_MSRpar.train.tsv",
#     "../data/2012_MSRpar.test.tsv",
#     "../data/2012_OnWN.test.tsv",
#     "../data/2013_OnWN.test.tsv",
#     "../data/2014_OnWN.test.tsv",
#     "../data/2014_images.test.tsv.txt",
#     "../data/2015_images.test.tsv",
# ]

# sentences = []
# for d in data:
#     print(f"path: {d}")
#     pl_data = pl.read_csv(d, separator="\t", has_header=False, quote_char=None, ignore_errors=True)
#     sentences.extend(pl_data['column_1'].to_list())
#     sentences.extend(pl_data['column_2'].to_list())

# len(set(sentences))

In [15]:
sentences_df = pl.read_csv("../data/sentences.txt", separator="\n", quote_char=None, has_header=False)
sentences_df.head(2)

column_1
str
"""A group of four children danci…"
"""The Conference Board said its …"


In [16]:
sentences = sentences_df['column_1'].to_list()
print(len(sentences))
sentences[:5]

14504


['A group of four children dancing in a backyard.',
 'The Conference Board said its measure of business confidence, which had fallen to 53 in the first quarter of 2003, improved to 60 in the most recent second quarter.',
 'a person eating a meal, often in a restaurant',
 'When you crossed the line, you violated the constitutional right," said Charles Weisselberg, who teaches law at the University of California, Berkeley.',
 "Ross Garber, Rowland's legal counsel, said the governor would have no comment on the condo deal."]

In [5]:
sentences = [word for word in list(set(sentences)) if type(word) is str]
print(len(sentences))

14504


### Embedding Generation

In [17]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(14504, 768)

In [19]:
np.save("../data/sentence_embeddings.npy", sentence_embeddings)

In [20]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = np.load("../data/sentence_embeddings.npy")
sentence_embeddings.shape

(14504, 768)

### IndexFlatL2

In [7]:
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal

14504

In [11]:
# Then search given a query `xq` and number of nearest neigbors to return `k`.
k = 4
xq = model.encode(["Someone sprints with a football"])

In [12]:
%%time
D, I = index.search(xq, k)
print(I)

[[10382  7600  7312  4212]]
CPU times: total: 0 ns
Wall time: 12.5 ms


In [ ]:
rows = I[0].tolist()
print(*[sentences[idx] for idx in rows], sep="\n")

A little boy is kicking up his feet in the water.
Nobody is in front of the colorful building
A football player is running past an official carrying a football
Two girls are playing inside a jumper house


In [14]:
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(rows):
    vecs[i, :] = index.reconstruct(val)

vecs.shape

(4, 768)

### Partitioning The Index

In [ ]:
nlist = 50 # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)
print(index.is_trained)

# since we added clustering, we are going to train the index
index.train(sentence_embeddings)
print(index.is_trained)

index.add(sentence_embeddings)
print(index.ntotal)  # number of embeddings indexed

False

In [53]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 4586 10252 12465   190]]
CPU times: total: 0 ns
Wall time: 0 ns


In [54]:
rows = I[0].tolist()
print(*[sentences[idx] for idx in rows], sep="\n")

A group of football players is running in the field
A group of people playing football is running in the field
Two groups of people are playing football
A person playing football is running past an official carrying a football


In [73]:
%%time
faiss.cvar.indexIVF_stats.reset()
index.nprobe = 1
D, I = index.search(xq, k)
print("nq:", faiss.cvar.indexIVF_stats.nq)
print("nlist:", faiss.cvar.indexIVF_stats.nlist)
print("ndis:", faiss.cvar.indexIVF_stats.ndis)        # distance computations
print("nheap:", faiss.cvar.indexIVF_stats.nheap_updates)

nq: 1
nlist: 1
ndis: 272
nheap: 21
CPU times: total: 0 ns
Wall time: 0 ns


### Vector Reconstruction

In [74]:
# we can't directly recontsruct because there is no direct mapping between the original vectors
# and their index position due to the addition of the IVF step.
# so we first create the direct mappings
index.make_direct_map()
index.reconstruct(7460)[:5]

array([-0.44000933,  0.3376906 , -0.30594134,  0.21360028, -0.3306173 ],
      dtype=float32)

### Quantization

In [ ]:
nlist = 50 # how many cells
m = 8 # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d) # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)
print(index.is_trained)
index.train(sentence_embeddings)
print(index.is_trained)
index.add(sentence_embeddings)
print(index.ntotal)  # number of embeddings indexed


False
True
14504


In [92]:
%%time
index.nprobe = 10  # align to previous IndexIVFFlat nprobe value
D, I = index.search(xq, k)
print(I)

[[  190   399  8328 12465]]
CPU times: total: 0 ns
Wall time: 984 μs


## Nearest Neighbor Indexes for Similarity Search

### Indexes in Search

In [ ]:
# We will be using the Sift1M dataset
# http://corpus-texmex.irisa.fr/

import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [94]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('../data/sift.tar.gz', "r:gz")
tar.extractall()

C:\Users\mukes\AppData\Local\Temp\ipykernel_29264\2915867793.py:5: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


In [95]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [97]:
# data we will search through
xb = read_fvecs('../data/sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('../data/sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])
print(xq.shape, xb.shape)

(1, 128) (1000000, 128)


In [ ]:
d = 128  # dimensionality of Sift1M data
k = 10  # number of nearest neighbors to return

index = faiss.IndexFlatIP(d)
index.add(xb)
D, I = index.search(xq, k)

There are two primary approaches to make search faster:
* Reduce vector size - through dimensionality reduction or reducing the number of bits representing our vectors values.
* Reduce search scope - we can do this by clustering or organizing vectors into tree structures based on certain attributes, similarity, or distance - and restricting our search to closest clusters or filter through most similar branches.

### Locality Sensitive Hashing

In [ ]:
nbits = d*4  # resolution of bucketed vectors
# initialize index and add vectors
index = faiss.IndexLSH(d, nbits)
index.add(xb)
# and search
D, I = index.search(xq, k)

Our nbits argument refers to the 'resolution' of the hashed vectors. A higher value means greater accuracy at the cost of more memory and slower search speeds.

### Hierarchical Navigable Small World Graphs

In [ ]:
# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index = faiss.IndexHNSWFlat(d, M)
# set efConstruction and efSearch parameters
index.hnsw.efConstruction = ef_construction
index.hnsw.efSearch = ef_search
# add data to index
index.add(xb)

# search as usual
D, I = index.search(xq, k)

Here, we have three key parameters for modifying our index performance.
* M - the number of nearest neighbors that each vertex will connect to.
* efSearch - how many entry points will be explored between layers during the search.
* efConstruction - how many entry points will be explored when building the index.

M and efSearch have a larger impact on search-time - efConstruction primarily increases index construction time (meaning a slower index.add), but at higher M values and higher query volume we do see an impact from efConstruction on search-time too.

HNSW gives us great search-quality at very fast search-speeds - but there’s always a catch - HNSW indexes take up a significant amount of memory. Using an M value of 128 for the Sift1M dataset requires upwards of 1.6GB of memory. However, we can increase our other two parameters - efSearch and efConstruction with no effect on the index memory footprint.


## Locality Sensitive Hashing

## END